In [6]:
import pandas as pd

In [8]:
#load data
prod_df = pd.read_json("../data/processed/meta_digital_cameras.json.gz")
reviews_df = pd.read_json("../data/processed/reviews_digital_cameras.json.gz")   

In [13]:
# merge data (inner mode) 
data_df = pd.merge(left=prod_df, right=reviews_df, on='asin')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87205 entries, 0 to 87204
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     87205 non-null  object
 1   description  87205 non-null  object
 2   title        87205 non-null  object
 3   brand        87205 non-null  object
 4   main_cat     87205 non-null  object
 5   date         87205 non-null  object
 6   price        87205 non-null  object
 7   asin         87205 non-null  object
 8   overall      87205 non-null  int64 
 9   vote         87205 non-null  int64 
 10  text         87205 non-null  object
 11  summary      87205 non-null  object
 12  timestamp    87205 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 9.3+ MB


In [15]:
#head of merge data
data_df.head(10)

,category,description,title,brand,main_cat,date,price,asin,overall,vote,text,summary,timestamp
0,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,1,47,"I bought the Smile to have a handy, inexpensiv...",Not the best choice,950
1,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,1,24,It is absolutely useless. The pictures are gr...,This camera is HORRIBLE!,971
2,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,4,73,Bought one from Amazon and begun immediately t...,What do you need?,966
3,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,1,28,"I'm on a small budget this summer, so I bought...","Worse than I thought, but at that price...",960
4,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,3,109,My first experiences with AGFA cameras were in...,Not bad for the money,959
5,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,1,5,I bought two of these for my kids and they are...,Just awful; avoid at all costs,986
6,"[Electronics, Camera & Photo, Digital Cameras,...","[Digital camera 640x480 resolution, 2MB built ...",Agfa ePhoto SMILE 0.2MP Digital Camera,Agfa,Camera & Photo,"January 11, 2001",,B00000JBIA,1,0,It's a nice little TOY. That's all it is. It...,Sold it for $40...I think I ripped the guy off.,978
7,"[Electronics, Camera & Photo, Digital Cameras,...",[Boosts productivity while improving your imag...,Kodak DC240 1.2MP Digital Camera w/ 3x Optical...,Kodak,Camera & Photo,"September 4, 1999",,B00000JDEL,4,3,I really like mine. It takes good pictures. It...,Kodak 240,971
8,"[Electronics, Camera & Photo, Digital Cameras,...",[Boosts productivity while improving your imag...,Kodak DC240 1.2MP Digital Camera w/ 3x Optical...,Kodak,Camera & Photo,"September 4, 1999",,B00000JDEL,5,4,This is the best camera I have seen for the pr...,"Great Camera, Great Features, Easy Use",969
9,"[Electronics, Camera & Photo, Digital Cameras,...",[Boosts productivity while improving your imag...,Kodak DC240 1.2MP Digital Camera w/ 3x Optical...,Kodak,Camera & Photo,"September 4, 1999",,B00000JDEL,4,10,"I have been in photography for about 15 years,...",Great Camera for the price!,965
